# Agents


In this tutorial, we implement an AI agent from scratch in Python using an LLM API directly to gain a better understanding of what's happening under the hood.

We implement the `Agent()` class step-by-step by incorporating the following core components: 
1. LLM (and instructions) : this is the LLM powering the agent's reasoning and decision-making capabilities with explicit guidelines of how the agent should behave.
2. Memory: conversation history (short-term memory) and that agent uses to understand the current interaction.
3. Tools: external tools that the agent can use to complete tasks.

Component 1: LLM (and instructions)
At the core of every agent, you have a large language model (LLM) with tool use capabilities.

We will use Claude for Sonnet through the Anthropic API, but we can easily adjust the code to any other LLM API of our choice.


In [7]:
%%capture
%pip install -U anthropic python-dotenv

import anthropic
import os
from dotenv import load_dotenv


load_dotenv()

print(anthropic.__version__)


In [37]:
client = Anthropic(api_key=os.environ["ANTHROPIC_API_KEY"])
client.models.list()


SyncPage[ModelInfo](data=[ModelInfo(id='claude-haiku-4-5-20251001', created_at=datetime.datetime(2025, 10, 15, 0, 0, tzinfo=datetime.timezone.utc), display_name='Claude Haiku 4.5', type='model'), ModelInfo(id='claude-sonnet-4-5-20250929', created_at=datetime.datetime(2025, 9, 29, 0, 0, tzinfo=datetime.timezone.utc), display_name='Claude Sonnet 4.5', type='model'), ModelInfo(id='claude-opus-4-1-20250805', created_at=datetime.datetime(2025, 8, 5, 0, 0, tzinfo=datetime.timezone.utc), display_name='Claude Opus 4.1', type='model'), ModelInfo(id='claude-opus-4-20250514', created_at=datetime.datetime(2025, 5, 22, 0, 0, tzinfo=datetime.timezone.utc), display_name='Claude Opus 4', type='model'), ModelInfo(id='claude-sonnet-4-20250514', created_at=datetime.datetime(2025, 5, 22, 0, 0, tzinfo=datetime.timezone.utc), display_name='Claude Sonnet 4', type='model'), ModelInfo(id='claude-3-7-sonnet-20250219', created_at=datetime.datetime(2025, 2, 24, 0, 0, tzinfo=datetime.timezone.utc), display_name='C

In [38]:
class Agent:
    "A simple Agent that can answer questions"

    def __init__(self):
        self.client = Anthropic(api_key=os.environ["ANTHROPIC_API_KEY"])
        self.model = 'claude-haiku-4-5-20251001'
        self.system_message = "You are a helpful assistant that breaks down problems into steps and solves them systematically."

    def chat(self, message):
        "Process user message and return response"
        
        response = self.client.messages.create(
            model=self.model,
            max_tokens=1024,
            system=self.system_message,
            messages=[
                {"role": "user", "content": message}
                ],
            temperature=0.1,
        )
        return response

In [39]:
agent = Agent()
response = agent.chat("I have 4 apples. How many do you have?")
print(response.content[0].text)

I don't have any apples! I'm an AI assistant, so I don't possess physical objects.

But I can help you with questions about your apples, like:
- How many you'd have if you got more or gave some away
- How to divide them among people
- Recipes or storage tips
- Math problems involving them

Is there something specific you'd like to do with your 4 apples?


Now let's follow it up with a second message.

In [40]:
agent = Agent()
response = agent.chat("I ate one apple. How many are left ?")
print(response.content[0].text)


I don't have enough information to answer this question. To tell you how many apples are left, I would need to know:

**How many apples did you start with?**

For example:
- If you started with 5 apples and ate 1, you'd have **4 left**
- If you started with 10 apples and ate 1, you'd have **9 left**

Could you tell me the starting number of apples?


As you can see, the agent lacks the information from the first message. That’s why we need to give the agent access to the conversation history.


##  Component 2: (Conversation) Memory
Memory in Agents can take many forms, such as short-term and long-term memory, and memory management can become a complex topic. For the sake of this tutorial, let's just keep it simple and start with a basic short-term memory implementation.

Short-term memory gives the agent access to the conversation state to the conversation history to understand the current interaction. In its simplest form, the short-term memory is just a list of past `messages` between the `user` and the `assistant` (note that the longer the conversaton becomes, you will run into the context window limitations and need to implement a more sophisticated solution).
We implement a short-term memory by adding `messages` property where we store
- the user input with `{"role": "user", "content": message}`
- the response with `{"role": "assistant", "content": response}`


In [41]:
class Agent:
    "A simple Agent that can answer questions"

    def __init__(self):
        self.client = Anthropic(api_key=os.environ["ANTHROPIC_API_KEY"])
        self.model = 'claude-haiku-4-5-20251001'
        self.system_message = "You are a helpful assistant that breaks down problems into steps and solves them systematically."
        self.messages = []


    def chat(self, message):
        "Process user message and return response"
        # Store user input in short-term memory
        self.messages.append({"role": "user", "content": message})
        
        response = self.client.messages.create(
            model=self.model,
            max_tokens=1024,
            system=self.system_message,
            messages=self.messages,
            temperature=0.1,
        )
        # Store assistant's response in short-term memory
        self.messages.append({"role": "assistant", "content": response.content})
        return response

In [42]:
agent = Agent()

response = agent.chat("I have 4 apples. How many do you have?")
print(response.content[0].text)

response = agent.chat("I ate 1 apple. How many are left?")
print(response.content[0].text)

I don't have any apples! I'm an AI assistant, so I don't possess physical objects.

But I can help you with questions about your apples, like:
- How many you'd have if you got more or gave some away
- How to divide them among people
- Recipes or storage tips
- Math problems involving them

Is there something specific you'd like to do with your 4 apples?
You started with 4 apples and ate 1, so:

4 - 1 = **3 apples left**


As you can see, the agent is now able to hold a conversation and to reference previous information.

But what happens, if you task the agent with a little more complex math problem?

In [43]:
agent = Agent()

response = agent.chat("What is 157.09 * 493.89?")

print(response.content[0].text)

I'll multiply 157.09 × 493.89 step by step.

**Breaking it down:**

157.09 × 493.89

Let me calculate this:

157.09 × 493.89 = **77,516.0301**

**Verification using the standard algorithm:**
- 157.09 has 2 decimal places
- 493.89 has 2 decimal places
- Total decimal places in answer: 4

157.09 × 493.89 ≈ **77,516.03** (rounded to 2 decimal places)

The answer is **77,516.0301** or approximately **77,516.03**


The agent’s answer sounds perfectly believable but if you validate it, you can actually see that even powerful LLMs like Claude 4 Sonnet can still make arithmetic errors without tools.


In [ ]:
157.09 * 493.89


77585.1801

# Component 3: Tool Use
To extend the agent's capabilities, we can provide it with tools that can range from simple functions to using external APIs. For this tutorial, we'll implement a simple `CalculatorTool` class that can handle math problems.

The exact implementation of tool use is different across providers, but at the core, always requires two key components:
- Function implementation: which is the actual function that executes the tool's logic, such as performing a calculation or making an API call.
- Tool schema: a structured description of the tool. The tool description is important because it tells the LLM what this tool does, when to use it, and what parameters it takes.

In [2]:
class CalculatorTool():
    """A tool for performing mathematical calculations"""

    def get_schema(self):
        return {
            "name": "calculator",
            "description": "Performs basic mathematical calculations, use also for simple additions",
            "input_schema": {
                "type": "object",
                "properties": {
                    "expression": {
                        "type": "string",
                        "description": "Mathematical expression to evaluate (e.g., '2+2', '10*5')"
                    }
                },
                "required": ["expression"]
            }
        }

    def execute(self, expression):
        """
        Evaluate mathematical expressions.
        WARNING: This tutorial uses eval() for simplicity but it is not recommended for production use.

        Args:
            expression (str): The mathematical expression to evaluate
        Returns:
            float: The result of the evaluation
        """
        try:
            result = eval(expression)
            return {"result": result}
        except:
            return {"error": "Invalid mathematical expression"}

In this tutorial, we are just implementing a single tool. In production code, you typically use an abstract base class to ensure a consistent interface across the tools.

In [3]:
calculator_tool = CalculatorTool()

calculator_tool.execute("157.09 * 493.89")

{'result': 77585.1801}

Now that we have a `CalculatorTool`, let's add tool use capabilities to our agent in the next three steps.
1.

In [26]:
from anthropic import Anthropic
class Agent:
    "A simple Agent that can answer questions"

    def __init__(self, tools):
        self.client = Anthropic(api_key=os.environ["ANTHROPIC_API_KEY"])
        self.model = 'claude-haiku-4-5-20251001'
        self.system_message = "You are a helpful assistant that breaks down problems into steps and solves them systematically."
        self.messages = []
        self.tools = tools
        self.tool_map = {tool.get_schema()["name"]: tool for tool in tools}


    def _get_tool_schemas(self):
        """Get the tool schema from the tools"""
        return [tool.get_schema() for tool in self.tools]

    def chat(self, message):
        "Process user message and return response"
        # Store user input in short-term memory
        self.messages.append({"role": "user", "content": message})
        
        response = self.client.messages.create(
            model=self.model,
            max_tokens=1024,
            system=self.system_message,
            tools=self._get_tool_schemas() if self.tools else None,
            messages=self.messages,
            temperature=0.1,
        )
        # Store assistant's response in short-term memory
        self.messages.append({"role": "assistant", "content": response.content})
        return response

In [27]:
calculator_tool = CalculatorTool()
agent = Agent(tools=[calculator_tool])

response = agent.chat("What is 157.09 * 493.89?")

for block in response:
  print(block)
  

('id', 'msg_01XNG4K8cWnP9ALUs6Q7imKG')
('content', [ToolUseBlock(id='toolu_01RjtcyeLSVEFJASHUN6snB2', input={'expression': '157.09 * 493.89'}, name='calculator', type='tool_use')])
('model', 'claude-haiku-4-5-20251001')
('role', 'assistant')
('stop_reason', 'tool_use')
('stop_sequence', None)
('type', 'message')
('usage', Usage(cache_creation=CacheCreation(ephemeral_1h_input_tokens=0, ephemeral_5m_input_tokens=0), cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=616, output_tokens=60, server_tool_use=None, service_tier='standard'))


In [28]:
import json

def run_agent(user_input, max_turns=10):
  calculator_tool = CalculatorTool()
  agent = Agent(tools=[calculator_tool])

  i = 0

  while i < max_turns: # It's safer to use max_turns rather than while True
    i += 1
    print(f"\nIteration {i}:")

    print(f"User input: {user_input}")
    response = agent.chat(user_input)
    print(f"Agent output: {response.content[0].text}")

    # Handle tool use if present
    if response.stop_reason == "tool_use":

        # Process all tool uses in the response
        tool_results = []
        for content_block in response.content:
            if content_block.type == "tool_use":
                tool_name = content_block.name
                tool_input = content_block.input

                print(f"Using tool {tool_name} with input {tool_input}")

                # Execute the tool
                tool = agent.tool_map[tool_name]
                tool_result = tool.execute(**tool_input)

                tool_results.append({
                    "type": "tool_result",
                    "tool_use_id": content_block.id,
                    "content": json.dumps(tool_result)
                })
                print(f"Tool result: {tool_result}")

        # Add tool results to conversation
        user_input = tool_results
    else:
      return response.content[0].text

  return

In [29]:
response = run_agent("If my brother is 32 years younger than my mother and my mother is 30 years older than me and I am 20, how old is my brother?")



Iteration 1:
User input: If my brother is 32 years younger than my mother and my mother is 30 years older than me and I am 20, how old is my brother?
Agent output: I need to work through this step-by-step.

Given information:
- I am 20 years old
- My mother is 30 years older than me
- My brother is 32 years younger than my mother

Let me calculate:
Using tool calculator with input {'expression': '20 + 30'}
Tool result: {'result': 50}
Using tool calculator with input {'expression': '50 - 32'}
Tool result: {'result': 18}

Iteration 2:
User input: [{'type': 'tool_result', 'tool_use_id': 'toolu_01FZhbohdMxtnwuTt6c918ST', 'content': '{"result": 50}'}, {'type': 'tool_result', 'tool_use_id': 'toolu_01DdYbdUjS29hWYheXPaVuGp', 'content': '{"result": 18}'}]
Agent output: **Your brother is 18 years old.**

Here's the breakdown:
1. You are 20 years old
2. Your mother is 30 years older than you: 20 + 30 = **50 years old**
3. Your brother is 32 years younger than your mother: 50 - 32 = **18 years o